In [ ]:
# Import packages 
import pandas as pd
import os
import yaml

# Local modules
from module.data_loader import OsirisData
from module.dataset import *
from module.modeling import randomforestregressormodel_train, lassoregressionmodel_train, supportvectormachinemodel_train
from module.modeling import randomforestregressormodel_pred, lassoregressionmodel_pred, supportvectormachinemodel_pred
from module.features import *
from module.settings import load_settings

In [2]:
# Choose which settings to load
settings_type = 'custom'  # [default,custom] Change to 'custom' to load custom settings
dataloader = "db" # Change to 'file' for file input 

In [ ]:
# class SettingsObject(dict):
#     # __class__ = dict
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         for key, value in kwargs.items():
#             if isinstance(value, dict):
#                 # Recursively convert nested dictionaries to Struct objects
#                 setattr(self, key, SettingsObject(**value))
#             else:
#                 setattr(self, key, value) # set attribute
#         # self.__class__ = dict
#     def __repr__(self):
#         return str(self.__dict__)
    
#     def __str__(self):
#         return str(self.__dict__)
    
#     def as_dict(self):
#         return self.__dict__


# def load_settings(config_file, settings_type = "default"):
#     with open(config_file, 'r') as file:
#         config = yaml.safe_load(file)
#     if settings_type == 'default':
#         settings = config['default_settings']
#     elif settings_type == 'custom':
#         settings = config['custom_settings']
#     else:
#         raise ValueError("No settings found. Choose 'default' or 'custom'.")
#     return SettingsObject(**settings)

In [ ]:
config_file = 'config.yaml'
settings = load_settings(config_file, settings_type)
print(settings)

True

In [5]:
# Check if train.csv and pred.csv exist in user_data folder, otherwise load synthetic datasests
# When user data is loaded and an error occurs here, please check if the sep = '\t' needs to be changed in the config.yaml file to another separator like ',' or '.'
# This should be the same as the separator used in your .csv file

if dataloader == 'file':
    if os.path.exists(settings.user_data_dir_train) and os.path.exists(settings.user_data_dir_pred):
        train_path,pred_path = user_data_dir_train, user_data_dir_pred
        print ('User datasets found')
    else:
        train_path,pred_path = settings.synth_data_dir_train, settings.synth_data_dir_pred
        print ('Pre-uploaded synthetic datasets found')
        
    train_df = pd.read_csv(train_path, sep = separator, engine='python')
    pred_df = pd.read_csv(pred_path, sep = separator, engine='python')
elif dataloader == 'db':
    pred_cohort = 2024
    train_cohort_min = 2023
    db = OsirisData(env_path="./")
    train_df = db.get_dataset(team="BB",min_cohort=train_cohort_min,max_cohort=pred_cohort-1)
    pred_df = db.get_dataset(team="BB",min_cohort=pred_cohort,max_cohort=pred_cohort)

__ Init done __
inschrijvingen geladen...
verzuimdata geladen...
vooropleidingdata geladen...
inschrijvingen geladen...
verzuimdata geladen...
vooropleidingdata geladen...


In [6]:
# Basic data cleaning: drop rows that are duplicate and change any NA values to the average value of the column it's in. 
train_basic_cl = basic_cleaning (train_df)
pred_basic_cl = basic_cleaning (pred_df)

In [7]:
# Detect if there are columns in which all rows have the same value and delete these columns from the train and predict datasets 
train_cleaned, pred_cleaned = remove_single_value_columns(train_basic_cl, pred_basic_cl)

In [ ]:
train_cleaned[settings.studentnumber_column]=train_cleaned[settings.studentnumber_column].astype("int")
pred_cleaned[settings.studentnumber_column]=pred_cleaned[settings.studentnumber_column].astype("int")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STUDENTNUMMER          87 non-null     int64  
 1   STUD_GENDER_M          87 non-null     int64  
 2   STUD_GENDER_V          87 non-null     int64  
 3   LEEFTIJD               87 non-null     int64  
 4   VOOROPLNIVEAU_NAN      87 non-null     int64  
 5   VOOROPLNIVEAU_VMBO_BB  87 non-null     int64  
 6   VOOROPLNIVEAU_VMBO_GL  87 non-null     int64  
 7   VOOROPLNIVEAU_VMBO_KB  87 non-null     int64  
 8   VOOROPLNIVEAU_VMBO_TL  87 non-null     int64  
 9   VOOROPLNIVEAU_MBO      87 non-null     int64  
 10  TOTAALAANW3            87 non-null     int64  
 11  TOTAALAANW6            87 non-null     int64  
 12  TOTAALAANW10           87 non-null     int64  
 13  AANW_PCT3              87 non-null     float64
 14  AANW_PCT6              87 non-null     float64
 15  AANW_PCT

In [9]:
# Apply function that changes categorical data into numerical data so it can be used as input for the models 
train_processed, pred_processed = convert_categorical_to_dummies(train_cleaned, pred_cleaned, settings.dropout_column, settings.separator)

In [10]:
# Use the function standardize_min_max to standardize the train and pred datasets using a min max scaler and save them as .csv files in the folder data/interim 
# These datasets can be used for the lasso and svm models, because reggression is sensitive to scaling 
train_df_sdd, pred_df_sdd = standardize_dataset(train_processed, pred_processed, settings.dropout_column, settings.separator)

In [30]:
# Code checks if retrain_models = True or False in config.yaml file. 
# When using your own datasets, change retrain_models in the config.yaml file to True, so the models are trained on your own data. 
# Warning: training the models can take a long time depending on the size and contents of your data. 
if settings.retrain_models == True:
    print ('Training models on the data...')
    best_rf_model = randomforestregressormodel_train(
        train_processed, settings.random_seed
        , settings.dropout_column, settings.rf_parameters)
    best_lasso_model = lassoregressionmodel_train(
        train_df_sdd, settings.random_seed
        , settings.dropout_column, settings.lasso_parameters)
    best_svm_model = supportvectormachinemodel_train(
        train_df_sdd, settings.random_seed
        , settings.dropout_column, settings.svm_parameters)
else:
    print('retrain_models is False in the config.yaml file, loading the pre-trained models')
# Folds = number of train/test splits of the dataset, candidates = models with different parameters and fits = folds * candidates

Training models on the data...
Fitting 5 folds for each of 486 candidates, totalling 2430 fits


KeyboardInterrupt: 

In [13]:
pred_processed.columns

Index(['STUDENTNUMMER', 'STUD_GENDER_M', 'STUD_GENDER_V', 'LEEFTIJD',
       'VOOROPLNIVEAU_NAN', 'VOOROPLNIVEAU_VMBO_BB', 'VOOROPLNIVEAU_VMBO_GL',
       'VOOROPLNIVEAU_VMBO_KB', 'VOOROPLNIVEAU_VMBO_TL', 'VOOROPLNIVEAU_MBO',
       'TOTAALAANW3', 'TOTAALAANW6', 'TOTAALAANW10', 'AANW_PCT3', 'AANW_PCT6',
       'AANW_PCT10', 'GEEN_UITVAL', 'dropout'],
      dtype='object')

In [ ]:
# Use the loaded models to predict on the datasets. 
# The lasso and SVM models use the standardized dataset ot predict an, but take the student numnbers from the 
# regular predict dataset. 
ranked_students_rf = randomforestregressormodel_pred (pred_processed, dropout_column, studentnumber_column)
ranked_students_lasso = lassoregressionmodel_pred(pred_df_sdd, pred_processed, settings.dropout_column, settings.studentnumber_column)
ranked_students_svm = supportvectormachinemodel_pred(pred_df_sdd, pred_processed, settings.dropout_column, settings.studentnumber_column)

In [ ]:
# Save the output files as either .xlsx or as three .csv files 
if settings.save_method == 'xlsx':
    writer = pd.ExcelWriter('models/predictions/ranked_students.xlsx', engine='xlsxwriter')
    ranked_students_rf.to_excel(writer, sheet_name='Random Forest', startrow=0, startcol=0, index=False)
    ranked_students_lasso.to_excel(writer, sheet_name='Lasso', startrow=0, startcol=0, index=False)
    ranked_students_svm.to_excel(writer, sheet_name='Support Vector Machine', startrow=0, startcol=0, index=False)
    writer.close()
    print ('Output file saved as .xlsx in the /models/predictions folder')
elif settings.save_method == 'csv':
    ranked_students_rf.to_csv('models/predictions/csv_output/ranked_students_rf.csv', sep='\t', index=False)
    ranked_students_lasso.to_csv('models/predictions/csv_output/ranked_students_lasso.csv', sep='\t', index=False)
    ranked_students_svm.to_csv('models/predictions/csv_output/ranked_students_svm.csv', sep='\t', index=False)
    print ('Output files saved as .csv in the /models/predictions/csv_output folder')
else:
    print('Invalid save method. For save_method in the config.yaml file, please fill in "xlsx" or "csv"')

NameError: name 'save_method' is not defined